In [1]:
import pandas as pd
from pandas import read_excel
from sqlalchemy import create_engine
from config import password
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
csv_file = "TA_restaurants_curated.csv"
restaurant_data_df = pd.read_csv(csv_file)
restaurant_data_df.head()

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170


In [3]:
#create new data with select columns
new_restaurant_data_df = restaurant_data_df[['Name','City',  'Rating', 'Number of Reviews']].copy()
new_restaurant_data_df.head()

,Name,City,Rating,Number of Reviews
0,Martine of Martine's Table,Amsterdam,5.0,136.0
1,De Silveren Spiegel,Amsterdam,4.5,812.0
2,La Rive,Amsterdam,4.5,567.0
3,Vinkeles,Amsterdam,5.0,564.0
4,Librije's Zusje Amsterdam,Amsterdam,4.5,316.0


In [4]:
#filter out only London restaurants
london_restaurant_data_df = new_restaurant_data_df.loc[new_restaurant_data_df["City"]=="London"]
london_restaurant_data_df.head()

,Name,City,Rating,Number of Reviews
45580,R & H cafe gallery,London,5.0,146.0
45581,Osteria Romana,London,5.0,112.0
45582,The Oystermen Seafood Bar & Kitchen,London,5.0,255.0
45583,Liman Restaurant,London,5.0,945.0
45584,Holy Smoke,London,5.0,433.0


In [5]:
#grab only ratings of 5.0 
london_restaurant_rated_df = london_restaurant_data_df.loc[london_restaurant_data_df["Rating"]==5.0]
london_restaurant_rated_df.head()

,Name,City,Rating,Number of Reviews
45580,R & H cafe gallery,London,5.0,146.0
45581,Osteria Romana,London,5.0,112.0
45582,The Oystermen Seafood Bar & Kitchen,London,5.0,255.0
45583,Liman Restaurant,London,5.0,945.0
45584,Holy Smoke,London,5.0,433.0


In [6]:
#drop index column
restaurant_df = london_restaurant_rated_df.reset_index(drop=True)
restaurant_df.head()

,Name,City,Rating,Number of Reviews
0,R & H cafe gallery,London,5.0,146.0
1,Osteria Romana,London,5.0,112.0
2,The Oystermen Seafood Bar & Kitchen,London,5.0,255.0
3,Liman Restaurant,London,5.0,945.0
4,Holy Smoke,London,5.0,433.0


In [7]:
restaurant_df_final = restaurant_df.drop("City", axis=1)
restaurant_df_final.head()

,Name,Rating,Number of Reviews
0,R & H cafe gallery,5.0,146.0
1,Osteria Romana,5.0,112.0
2,The Oystermen Seafood Bar & Kitchen,5.0,255.0
3,Liman Restaurant,5.0,945.0
4,Holy Smoke,5.0,433.0


In [22]:
restaurant_final = restaurant_df_final.rename(columns={"Name":"Restaurant_Name", "Rating":"Review_Rating", 
                                                       "Number of Reviews":"Number_of_Reviews"})
restaurant_final.head()

,Restaurant_Name,Review_Rating,Number_of_Reviews
0,R & H cafe gallery,5.0,146.0
1,Osteria Romana,5.0,112.0
2,The Oystermen Seafood Bar & Kitchen,5.0,255.0
3,Liman Restaurant,5.0,945.0
4,Holy Smoke,5.0,433.0


In [23]:
#export final restaurant dataframe to csv
restaurant_final.to_csv("restaurant_final.csv")

In [24]:
#import excel file
excel_file = "London_hotel_reviews.xlsx"
london_hotel_data_df = pd.read_excel(excel_file)
london_hotel_data_df.head()

,Property Name,Review Rating,Location,Review Title,Review Text,Location Of The Reviewer,Date Of Review
0,Apex London Wall Hotel,5.0,London,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...,"Casale Monferrato, Italy",2012-10-20
1,Corinthia Hotel London,5.0,London,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,"Savannah, Georgia",2016-03-23
2,The Savoy,5.0,London,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,London,2013-07-30
3,Rhodes Hotel,4.0,London,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,"Maui, Hawaii",2012-06-02
4,The Savoy,5.0,London,Perfection,Form the moment we arrived until we left we ex...,"London, United Kingdom",2017-11-24


In [25]:
#create new data with select columns
new_london_hotel_df = london_hotel_data_df[['Property Name','Location','Review Rating']].copy()
new_london_hotel_df.head()

,Property Name,Location,Review Rating
0,Apex London Wall Hotel,London,5.0
1,Corinthia Hotel London,London,5.0
2,The Savoy,London,5.0
3,Rhodes Hotel,London,4.0
4,The Savoy,London,5.0


In [26]:
#grab only ratings of 5.0 
london_hotel_rated_df = new_london_hotel_df.loc[new_london_hotel_df["Review Rating"]==5.0]
london_hotel_rated_df.head()

,Property Name,Location,Review Rating
0,Apex London Wall Hotel,London,5.0
1,Corinthia Hotel London,London,5.0
2,The Savoy,London,5.0
4,The Savoy,London,5.0
6,Apex London Wall Hotel,London,5.0


In [36]:
#group dataframe by "Property Name"
london_hotel_reviews_group = london_hotel_rated_df.groupby("Property Name")
london_hotel_reviews_group.head()

SyntaxError: invalid syntax (<ipython-input-36-f39437ef5fb3>, line 2)

In [28]:
#do a count to get number of reviews per hotel
#london_hotel_reviews_count["Number of Reviews"] = london_hotel_reviews_group.groupby(["Property Name"]).size()
#london_hotel_reviews_count

#Count number of reviews per hotel
london_hotel_reviews_count = pd.DataFrame(london_hotel_reviews_group["Review Rating"].value_counts())
london_hotel_reviews_count.head()

,,Review Rating
Property Name,Review Rating,
45 Park Lane - Dorchester Collection,5.0,157
A To Z Hotel,5.0,10
Apex London Wall Hotel,5.0,1639
"Bulgari Hotel, London",5.0,384
Corinthia Hotel London,5.0,2442


In [32]:
#rename the "Property Name" column to "Number of Reviews"
hotel_final_semifinal = london_hotel_reviews_count.rename(columns={"Review Rating":"Number_of_Reviews"})
hotel_final_semifinal.head()

,,Number_of_Reviews
Property Name,Review Rating,
45 Park Lane - Dorchester Collection,5.0,157
A To Z Hotel,5.0,10
Apex London Wall Hotel,5.0,1639
"Bulgari Hotel, London",5.0,384
Corinthia Hotel London,5.0,2442


In [33]:
hotel_final = london_hotel_reviews_count.rename(columns={"Property Name":"Property_Name", "Review Rating":"Review_Rating"})
hotel_final.head()

,,Review_Rating
Property Name,Review Rating,
45 Park Lane - Dorchester Collection,5.0,157
A To Z Hotel,5.0,10
Apex London Wall Hotel,5.0,1639
"Bulgari Hotel, London",5.0,384
Corinthia Hotel London,5.0,2442


In [16]:
#export final hotel dataframe to csv
hotel_final.to_csv("hotel_final.csv")

In [17]:
#Connect to local database
rds_connection_string = "root:" + password + "@127.0.0.1/london_fivestar"
engine = create_engine(f'mysql://{rds_connection_string}')

In [18]:
engine.table_names()

['hotel_final', 'london_fivestar', 'restaurant_final']

In [21]:
#Use pandas to load csv converted DataFrame into database
#restaurant_final.to_sql(schema='london_fivestar', con=engine, if_exists='append', index=False)
restaurant_final.to_sql("restaurant_final",schema="london_fivestar", con=engine, if_exists="append", index=False)

InternalError: (pymysql.err.InternalError) (1054, "Unknown column 'Restaurant Name' in 'field list'") [SQL: 'INSERT INTO london_fivestar.restaurant_final (`Restaurant Name`, `Review Rating`, `Number of Reviews`) VALUES (%(Restaurant Name)s, %(Review Rating)s, %(Number of Reviews)s)'] [parameters: ({'Restaurant Name': 'R & H cafe gallery', 'Review Rating': 5.0, 'Number of Reviews': 146.0}, {'Restaurant Name': 'Osteria Romana', 'Review Rating': 5.0, 'Number of Reviews': 112.0}, {'Restaurant Name': 'The Oystermen Seafood Bar & Kitchen', 'Review Rating': 5.0, 'Number of Reviews': 255.0}, {'Restaurant Name': 'Liman Restaurant', 'Review Rating': 5.0, 'Number of Reviews': 945.0}, {'Restaurant Name': 'Holy Smoke', 'Review Rating': 5.0, 'Number of Reviews': 433.0}, {'Restaurant Name': 'The Clink Restaurant', 'Review Rating': 5.0, 'Number of Reviews': 588.0}, {'Restaurant Name': 'Bar 61 Restaurant', 'Review Rating': 5.0, 'Number of Reviews': 1730.0}, {'Restaurant Name': 'Taste Of Nawab', 'Review

In [ ]:
hotel_final.to_sql(name='london_fivestar', con=engine, if_exists='append', index=False)